In [14]:
!pip install langchain langchain-huggingface sentence-transformers faiss-cpu requests

In [15]:
pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [16]:
import requests
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [ ]:
# Step 1: Load and Split Articles from URLs
urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
]

loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

# Split large documents into manageable chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)
print(f"✅ Split into {len(docs)} chunks")

✅ Split into 32 chunks


In [18]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_index = FAISS.from_documents(docs, embedding_model)

In [21]:
# Step 3: Define a function to query LM Studio with retrieved context
def query_lmstudio_rag(prompt, retrieved_docs, model="deepseek-r1-distill-qwen-7b"):
    url = "http://localhost:1234/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    # Combine context from retrieved docs
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Construct final prompt with retrieved context
    final_prompt = f"""Use the following news article excerpts to answer the question.

{context}

Question: {prompt}
Answer:"""

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": final_prompt}],
        "temperature": 0.9,
        "max_tokens": 500
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        if "choices" not in result:
            print("❌ Unexpected API response format:")
            print(result)
            return "Error: 'choices' not found in API response."

        return result["choices"][0]["message"]["content"]

    except requests.exceptions.RequestException as e:
        return f"❌ API request failed: {e}"


In [ ]:
# Step 4: Ask a question (retrieval + generation)
user_question = "What is price of the Tiago iCNG?"

# Retrieve top 4 most relevant document chunks
retrieved_docs = vector_index.similarity_search(user_question, k=4)

# Get answer from DeepSeek via LM Studio
response = query_lmstudio_rag(user_question, retrieved_docs)
print("\n📢 LLM Response:\n", response)

In [ ]:
def run_rag_query(question: str, k=4):
    # Step 1: Retrieve top-k relevant document chunks
    retrieved_docs = vector_index.similarity_search(question, k=k)

    # Step 2: Ask LM Studio with the retrieved documents
    answer = query_lmstudio_rag(question, retrieved_docs)
    
    # Optional: also return the sources
    sources = [doc.metadata.get("source", "Unknown") for doc in retrieved_docs]
    
    return {
        "answer": answer,
        "sources": sources
    }

In [31]:
question = "What is price of the Tiago iCNG?"
response = run_rag_query(question)

# Print final output
print("📢 Answer:\n", response["answer"])
print("\n📚 Sources:")
for i, src in enumerate(response["sources"], 1):
    print(f"{i}. {src}")


📢 Answer:
 <think>
Okay, so I need to figure out the price of the Tiago iCNG based on the news article provided. Let me read through the excerpt carefully to extract all relevant information.

The article starts by mentioning two models from Tata Motors: the Tiago iCNG and the Tigor iCNG. The pricing for these vehicles is discussed right after, so I should look for those numbers. 

Looking at the first part of the excerpt:

"The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh."

So it looks like the Tiago iCNG starts at Rs 6.55 lakh and goes up to Rs 8.1 lakh. The user is specifically asking for the price of the Tiago iCNG, so I need to provide both the minimum and maximum price range.

I should double-check if there's any other mention of the Tiago iCNG in the article that could affect this pricing. Scanning further down, I see more details about the Punch iCNG but no additional information on the Ti